# MNIST Digits Recognition System : Neural Networks

### import 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

### Creating Fully Connected Neural Network

In [31]:
class NN(nn.Module):
    # initial declaration function
    def __init__(self, input_size, num_classes):
        # calling initialization method of super class
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        

In [32]:
model = NN(784, 10)
x = torch.randn(64, 784)
print(model(x).shape)

torch.Size([64, 10])


In [33]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Hyper parameters

In [43]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

### Load Data

In [44]:
# Train dataset
train_data = datasets.MNIST(root="**/Dataset/", train=True, transform=transforms.ToTensor(), download=False)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
# Test dataset
test_data = datasets.MNIST(root="**/Dataset/", train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

### Initialize Network

In [45]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

### Loss and Optimizer

In [46]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Train Network

In [47]:
for epoch in range(num_epochs):
    print(f"Epoch: {epoch+1}")
    for batch_idx, (data, target) in enumerate(train_loader):
        # Get Data to Cuda
        data = data.to(device=device)
        target = target.to(device=device)
        # Reshape to vectorize
        data = data.reshape(data.shape[0], -1) # vectorize [28,28] to 784

        # forward
        scores = model(data)
        loss = criterion(scores, target)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10


### Accuracy on training  & test dataset

In [48]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
        
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)

        print(f'Number of Correct: {num_correct}')
        print(f'Number of Samples: {num_samples}')
        accuracy = float(num_correct)/float(num_samples) * 100
        print(f'Accuracy: {accuracy:.2f}')

    # model.train()
    # return acc

In [49]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Number of Correct: 59110
Number of Samples: 60000
Accuracy: 98.52
Checking accuracy on test data
Number of Correct: 9714
Number of Samples: 10000
Accuracy: 97.14
